# Webis Abstractive Snippet Corpus 2020

https://zenodo.org/record/3653834#.Y4fccDPMIc4

- prepared by MaxM
- paper: [Abstractive Snippet Generation](https://arxiv.org/abs/2002.10782)

Unfortunately the data quality is very bad, so we will probably not use it.

In [12]:
from pathlib import Path
import json
import pandas as pd

import pyarrow.parquet as pq
import pyarrow as pa

print(Path.cwd())

fn = Path('../webis/webis_grouped.parquet')

print('reading: ', fn)
pf = pq.ParquetFile(fn)
print('num_row_groups:', pf.num_row_groups)


/media/koepf/data2/laion/laion_sum_data
reading:  ../webis/webis_grouped.parquet
num_row_groups: 359


In [22]:
chunk_size = 400000
prefix = 'webis'
compression = 'snappy'

output_dir = Path('data/webis')
output_dir.mkdir(parents=True, exist_ok=True)

def write_part(df, output_dir, prefix, part, compression):
    fn = output_dir / f'{prefix}_{part:05d}.{compression}.parquet'
    print(f'writing part {part}: {len(df)} rows')
    df.to_parquet(fn, compression=compression, engine="pyarrow", row_group_size=1000)

frames = []
row_count = 0

part = 0
for x in pf.iter_batches(batch_size=10000):
    df_part = x.to_pandas()
    provenance = pd.array([json.dumps({ 'src': prefix, 'doi': '10.5281/zenodo.3653834' })] * len(df_part), dtype="string")
    df_part["provenance"] = provenance
    row_count += len(df_part)
    frames.append(df_part)

    if row_count >= chunk_size:
        df = pd.concat(frames, ignore_index=True)
        write_part(df, output_dir, prefix, part, compression)
        frames = []
        row_count = 0
        part += 1

if row_count > 0:
    df = pd.concat(frames, ignore_index=True)
    write_part(df, output_dir, prefix, part, compression)

    

writing part 0: 400000 rows
writing part 1: 400000 rows
writing part 2: 400000 rows
writing part 3: 400000 rows
writing part 4: 400000 rows
writing part 5: 400000 rows
writing part 6: 400000 rows
writing part 7: 400000 rows
writing part 8: 389701 rows


In [23]:
df = pd.read_parquet(path=str(output_dir / 'webis_00003.snappy.parquet'), columns=['text', 'summary'])
print(len(df))


400000


In [31]:
print('text:', df['text'][1500])
print('summary:', df['summary'][1500])

text: signature our commitment to our associates goes beyond just growing their professional skills . for many of our team members , working in one of our restaurants is often their first job . so we strive to develop their life skills foundational abilities such as teamwork , responsibility , problem solving , positive energy , and a relentless drive to serve and satisfy customers so they can grow to their full capability , be successful in whatever they do , and achieve their dreams . world hunger relief is our signature global program to address hunger and hopelessness . our partnership with the world food programme has succeeded in ways we never imagined possible . through it we have gone beyond making a contribution to relieving hunger and we have built momentum for ongoing volunteerism and community engagement globally by giving our associates the tools , opportunities and encouragement to place themselves at the heart of pressing societal issues . since world hunger relief movem